**GET DATA**

Summary of this notebook: ...

Todo:

1. Self supervised representation learning using colorization as a pretext task
2. Taking in an image I , rotating it by an a set angle A--> rotated image (I_A), rotated angle A (actual label) (Done)
3. Dataloader that does this somehow (Done)
4. Some model-> ResNet50  + new full-connected layer (feature vector-> probability that image belongs to one of n classes) (Done-ish)
5. Additional training with the above model minimizing the loss wrt. actual rotated angle
f(I_A)=min[L(A_predict,A)]
6. Save this new model

7. Proof of concept:
Imagenet-> Train on Caltech validation on caltech




Definition of Done: ...

# Imports


In [1]:
# Mount Google drive
from google.colab import drive
import os
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# Change to current dataset
os.chdir("/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code")

In [3]:
# Imports from Colab 2
import math
import numpy as np
import pickle
import keras
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Import model architecture
from tensorflow.keras.applications import VGG16

Using TensorFlow backend.


In [20]:
# Imports for Colab 6
import cv2 # Read raw image
import glob
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from scipy import ndimage # For rotation task or
import imutils
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.python.keras.utils import data_utils
from tensorflow.keras.preprocessing.image import Iterator


# Imports for Colorizer
from os import path
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, AveragePooling2D, MaxPooling2D, Reshape, Conv2DTranspose, ZeroPadding2D, Add
from tensorflow.keras.layers import Activation, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import random
import tensorflow as tf



In [5]:
# Check to see if GPU is being used
tensorflow.test.gpu_device_name()

''

# Data Augmentation/Analysis

In [6]:
# Raw dataset/path/model paths
dataset = "UCMerced_LandUse"
dataPath = ("Datasets/"+ dataset+ "/Splits_2")

train_image_globs=glob.glob(dataPath+'/train/**/*.tif')
val_image_globs=glob.glob(dataPath+'/val/**/*.tif')

# Dataloader creation and test

In [ ]:
dims=(256,256,3)

train_name='merced_xtrain.npy'
test_name='merced_xtest.npy'

if path.exists(train_name):
  print("Train exists")
  X_train=np.load(train_name)
else:
  X_train=np.empty((0,*dims))

  # for f in train_image_globs:
  #   image=np.expand_dims(img_to_array(load_img(f,target_size=dims))/255.,0)
  #   X_train=np.vstack((X_train,image))

  # with open(train_name, 'wb') as f:
  #   np.save(f,X_train,allow_pickle=True)
  

if path.exists(test_name):
  print("Test exists")
  X_test=np.load(test_name)
else:
  X_test=np.empty((0,*dims))

#   for f in val_image_globs:
#     image=np.expand_dims(img_to_array(load_img(f,target_size=dims))/255.,0)
#     X_test=np.vstack((X_test,image))

#   with open(test_name, 'wb') as f:
#     np.save(f,X_test,allow_pickle=True)

Train exists


In [ ]:
print(X_train.shape,X_test.shape)

In [ ]:
train_dataGenerator = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

batch_size = 16

In [ ]:
def batch_lab(batch_size,channel_pred):
  for batch in train_dataGenerator.flow(X_train,batch_size=batch_size):
    lab_batch=rgb2lab(batch)

    if channel_pred=="0":
      X_batch=lab_batch[:,:,:,[0]]
      Y_batch=lab_batch[:,:,:,[1,2]]
    elif channel_pred=="1_2":
      X_batch=lab_batch[:,:,:,[1,2]]
      Y_batch=lab_batch[:,:,:,[0]]
    yield X_batch,Y_batch

def custom_loss(y_true, y_pred):
  y_true_hist=tf.histogram_fixed_width(y_true, value_range=(-1., 1.), nbins=20) 
  y_pred_hist=tf.histogram_fixed_width( y_pred, value_range=(-1., 1.), nbins=20) 
  return K.mean(K.square(y_true_hist-y_pred_hist)) + y_pred*0 

# Model creation

In [21]:
# Model_0=VGG16(include_top=True,input_shape=(256,256,1),weights=None)
# Model_12=VGG16(include_top=True,input_shape=(256,256,2),weights=None)

def FCN8(input_height,input_width,input_channels,n_classes=32):
    ## input_height and width must be devisible by 32 because maxpooling with filter size = (2,2) is operated 5 times,
    ## which makes the input_height and width 2^5 = 32 times smaller
    # assert input_height%32 == 0
    # assert input_width%32 == 0
    if input_channels==1:
      divisor=3
    elif input_channels==2:
      divisor=(3/2)
    
    IMAGE_ORDERING =  "channels_last" 

    img_input = Input(shape=(input_height,input_width, input_channels)) ## Assume 224,224,3
    
    ## Block 1
    x = Conv2D(64//divisor, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format=IMAGE_ORDERING )(img_input)
    x = Conv2D(64//divisor, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=IMAGE_ORDERING )(x)
    f1 = x
    
    # Block 2
    x = Conv2D(128//divisor, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(128//divisor, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=IMAGE_ORDERING )(x)
    f2 = x

    # Block 3
    x = Conv2D(256//divisor, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256//divisor, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(256//divisor, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=IMAGE_ORDERING )(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=IMAGE_ORDERING )(x)
    pool3 = x

    # Block 4
    x = Conv2D(512//divisor, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512//divisor, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512//divisor, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=IMAGE_ORDERING )(x)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=IMAGE_ORDERING )(x)## (None, 14, 14, 512) 

    # Block 5
    x = Conv2D(512//divisor, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format=IMAGE_ORDERING )(pool4)
    x = Conv2D(512//divisor, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format=IMAGE_ORDERING )(x)
    x = Conv2D(512//divisor, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=IMAGE_ORDERING )(x)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=IMAGE_ORDERING )(x)## (None, 7, 7, 512)

    
    vgg  = Model(  img_input , pool5  )
    # vgg.load_weights(VGG_Weights_path) ## loading VGG weights for the encoder parts of FCN8
    
    n = 4096
    o = ( Conv2D( n , ( 7 , 7 ) , activation='relu' , padding='same', name="conv6", data_format=IMAGE_ORDERING))(pool5)
    conv7 = ( Conv2D( n , ( 1 , 1 ) , activation='relu' , padding='same', name="conv7", data_format=IMAGE_ORDERING))(o)
    
    
    ## 4 times upsamping for pool4 layer
    conv7_4 = Conv2DTranspose( n_classes , kernel_size=(4,4) ,  strides=(4,4) , use_bias=False, data_format=IMAGE_ORDERING )(conv7)
    ## (None, 224, 224, 10)
    ## 2 times upsampling for pool411
    pool411 = ( Conv2D( n_classes , ( 1 , 1 ) , activation='relu' , padding='same', name="pool4_11", data_format=IMAGE_ORDERING))(pool4)
    pool411_2 = (Conv2DTranspose( n_classes , kernel_size=(2,2) ,  strides=(2,2) , use_bias=False, data_format=IMAGE_ORDERING ))(pool411)
    
    pool311 = ( Conv2D( n_classes , ( 1 , 1 ) , activation='relu' , padding='same', name="pool3_11", data_format=IMAGE_ORDERING))(pool3)
        
    o = Add(name="add")([pool411_2, pool311, conv7_4 ])
    o = Conv2DTranspose( n_classes , kernel_size=(8,8) ,  strides=(8,8) , use_bias=False, data_format=IMAGE_ORDERING )(o)
    o = (Activation('softmax'))(o)
    
    model = Model(img_input, o)

    return model



In [22]:
model = FCN8(input_height=256,
             input_width=256,
             input_channels=1,
             n_classes=32)
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 21) 210         input_6[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 21) 3990        block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 21) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

# Model Training

In [ ]:
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
model.fit(batch_grayrgb(batch_size),
                    epochs=30,
                    steps_per_epoch=1680/16)


In [ ]:
model.save('Models/Colorization_CustomCNN_epoch30_ucmerced')

## Model Testing

In [ ]:
model=load_model('Models/Colorization_CustomCNN_epoch30_ucmerced')

In [ ]:
for i in range(10):
  index=np.random.randint(0,len(X_test))

  X_test_im=rgb2gray(np.expand_dims(X_test[index],0))
  X_test_im=np.repeat(np.expand_dims(X_test_im,3),3,axis=3)

  out_image=np.squeeze(model.predict(X_test_im))

  fig=plt.figure()
  plt.subplot(1, 3, 1)
  plt.imshow(X_test[index])
  plt.subplot(1,3,2)
  plt.imshow(np.squeeze(X_test_im))
  plt.subplot(1,3,3)
  plt.imshow(out_image)
  plt.show()

  
  

In [ ]:
X_test_m = rgb2lab(X_test)[:,:,:,0]
X_test_m = X_test_m.reshape(X_test_m.shape+(1,))
Y_test = rgb2lab(X_test)[:,:,:,1:]
Y_test = Y_test / 128
print(model.evaluate(X_test_m, Y_test, batch_size=batch_size))

In [ ]:
rand_index=np.random.randint(0,len(val_image_globs))

sample_image=img_to_array(load_img(val_image_globs[rand_index]))
sample_image=np.expand_dims(sample_image/255.,0)

sample_image_x=rgb2lab(sample_image)[:,:,:,0]
sample_image_x=sample_image_x.reshape(sample_image_x.shape+(1,))

# Test model
output = model.predict(sample_image_x)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = sample_image_x[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("Models/colorization_out/img_"+str(rand_index)+".png", lab2rgb(cur))
    imsave("Models/colorization_out/raw_img_"+str(rand_index)+".png",np.squeeze(sample_image))

